# Refinitiv Data Platform Library for Python
## Delivery - OMMItemStream - Market Price data via callback

This notebook demonstrates how to use the OMM Item Stream interface to request streaming Quote and Trade data

## Set the location of the configuration file
For ease of use, you can set various initialization parameters of the RD Library in the **_refinitiv-data.config.json_** configuration file - as described in the Quick Start -> Sessions example.

### One config file for the tutorials
As these tutorial Notebooks are categorised into sub-folders and to avoid the need for multiple config files, we will use the _RD_LIB_CONFIG_PATH_ environment variable to point to a single instance of the config file in the top-level ***Configuration*** folder.

Before proceeding, please **ensure you have entered your credentials** into the config file in the ***Configuration*** folder.

In [1]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../../Configuration"

In [2]:
from refinitiv.data.delivery import omm_stream
import refinitiv.data as rd
import datetime
import json

## Open the default session

To open the default session ensure you have a '*refinitiv-data.config.json*' in the ***Configuration*** directory, populated with your credentials and specified a 'default' session in the config file    


In [3]:
rd.open_session()

<refinitiv.data.session.platform.Definition object at 0x1b36bb9cd00 {session_name='rdp'}>

## Define a function to display events

In [4]:
# Callback function to display data or status events
def display_event(eventType, event):
    currentTime = datetime.datetime.now().time()
    print("----------------------------------------------------------")
    print(">>> {} event received at {}".format(eventType, currentTime))
    print(json.dumps(event, indent=2))
    return

## Create an Item Stream and register the event callbacks

In [5]:
stream = omm_stream.Definition(
    name="EUR=", 
    fields=['BID', 'ASK','OPEN_PRC','HST_CLOSE','TIMACT'], 
    domain='MarketPrice').get_stream()

# Refresh - the first full imaage we get back from the server
stream.on_refresh(lambda item_stream, event : display_event("Refresh", event))
# Update - as and when field values change, we receive updates from the server
stream.on_update(lambda item_stream, event : display_event("Update", event))
# Status - if data goes stale or item closes, we get a status message
stream.on_status(lambda item_stream, event : display_event("Status", event))
# Other errors
stream.on_error(lambda item_stream, event : display_event("Error", event))

## Open the ItemStream
The **open()** call to open the Item Stream is a synchronous one. This means the first event (either via on_refresh(), on_status() or on_error()) can occur before the open() method returns.   
However, if we use the **open_async()** asynchronous method instead, the first event callback will be after  open_async() returns.

In [6]:
# Send request to server and open stream
stream.open()
# We should receive the initial Refresh for the current field values
# followed by updates for the fields as and when they occur

----------------------------------------------------------
>>> Refresh event received at 12:16:01.118182
{
  "ID": 1,
  "Type": "Refresh",
  "Key": {
    "Service": "ELEKTRON_DD",
    "Name": "EUR="
  },
  "State": {
    "Stream": "Open",
    "Data": "Ok"
  },
  "Qos": {
    "Timeliness": "Realtime",
    "Rate": "JitConflated"
  },
  "PermData": "AwEBUmw=",
  "SeqNumber": 1902,
  "Fields": {
    "TIMACT": "12:16:00",
    "OPEN_PRC": 1.1206,
    "HST_CLOSE": 1.1206,
    "BID": 1.1275,
    "ASK": 1.1279
  }
}


<OpenState.Opened: 1>

----------------------------------------------------------
>>> Update event received at 12:16:01.349115
{
  "ID": 1,
  "Type": "Update",
  "UpdateType": "Unspecified",
  "DoNotConflate": true,
  "Key": {
    "Service": "ELEKTRON_DD",
    "Name": "EUR="
  },
  "SeqNumber": 1918,
  "Fields": {
    "BID": 1.1276,
    "ASK": 1.1278,
    "TIMACT": "12:16:00"
  }
}
----------------------------------------------------------
>>> Update event received at 12:16:01.349115
{
  "ID": 1,
  "Type": "Update",
  "UpdateType": "Unspecified",
  "DoNotConflate": true,
  "Key": {
    "Service": "ELEKTRON_DD",
    "Name": "EUR="
  },
  "SeqNumber": 1982,
  "Fields": {
    "BID": 1.1276,
    "ASK": 1.1277,
    "TIMACT": "12:16:00"
  }
}
----------------------------------------------------------
>>> Update event received at 12:16:01.849395
{
  "ID": 1,
  "Type": "Update",
  "UpdateType": "Unspecified",
  "DoNotConflate": true,
  "Key": {
    "Service": "ELEKTRON_DD",
    "Name": "EUR="
  },
  "SeqNumber": 204

## Close Stream

In [7]:
stream.close()

<OpenState.Closed: 3>

## Close the session

In [9]:
rd.close_session()